In [1]:
import pandas as pd
import os

def ler_dados_excel(caminho_arquivo, aba):
    # Ler o arquivo Excel usando o Pandas
    df = pd.read_excel(caminho_arquivo, sheet_name=aba)
    return df

def analisar_dados(df, pessoa):
    # Filtrar apenas os dados da pessoa (PF ou PJ)
    df_pessoa = df[df['PESSOA'] == pessoa]

    # Número de beneficiários ativos
    num_beneficiarios_ativos = df_pessoa[df_pessoa['SITUACAO'] == 'Ativo']['MATRICULA'].nunique()

    # Valor da receita
    valor_receita = df_pessoa[df_pessoa['SITUACAO'] == 'Ativo']['Mensalidade'].sum()

    # Valor das despesas
    valor_despesas = df_pessoa[df_pessoa['SITUACAO'] == 'Ativo']['Despesas'].sum()

    return num_beneficiarios_ativos, valor_receita, valor_despesas

def analisar_dados_regiao(df, pessoa):
    regioes = df['regiao'].unique()
    dados_regioes = []

    for regiao in regioes:
        df_regiao = df[df['regiao'] == regiao]
        num_beneficiarios_ativos, valor_receita, valor_despesas = analisar_dados(df_regiao, pessoa)
        dados_regioes.append((regiao, num_beneficiarios_ativos, valor_receita, valor_despesas))

    return dados_regioes

def main():
    # Se estiver em um ambiente interativo, use o diretório atual como diretório do código
    if '__file__' not in globals():
        diretorio_codigo = os.getcwd()
    else:
        # Caso contrário, use o diretório do arquivo atual como diretório do código
        diretorio_codigo = os.path.dirname(os.path.abspath(__file__))

    # Caminho relativo para o arquivo analitixa-pf.xlsx dentro do diretório do código
    caminho_arquivo = os.path.join(diretorio_codigo, 'analitica.xlsx')

    abas = ['Janeiro-ANALITICO 2023', 'Fevereiro-ANALITICO 2023', 'Março-ANALITICO 2023',
            'Abril-ANALITICO 2023', 'Maio-ANALITICO 2023', 'Junho-ANALITICO 2023']

    dados_pf = []
    dados_pj = []

    for aba in abas:
        df = ler_dados_excel(caminho_arquivo, aba)

        if df is not None and not df.empty:
            # Analisar os dados para PF e PJ e obter as informações por região
            dados_pf_regiao = analisar_dados_regiao(df, 'PF')
            dados_pj_regiao = analisar_dados_regiao(df, 'PJ')

            mes = aba.split('-')[0]

            # Adicionar as informações para PF na lista de dados de PF
            for regiao, num_beneficiarios_ativos_pf, valor_receita_pf, valor_despesas_pf in dados_pf_regiao:
                dados_pf.append((mes, regiao, num_beneficiarios_ativos_pf, valor_receita_pf, valor_despesas_pf))

            # Adicionar as informações para PJ na lista de dados de PJ
            for regiao, num_beneficiarios_ativos_pj, valor_receita_pj, valor_despesas_pj in dados_pj_regiao:
                dados_pj.append((mes, regiao, num_beneficiarios_ativos_pj, valor_receita_pj, valor_despesas_pj))

    # Criar a planilha com a aba "PF"
    df_pf = pd.DataFrame(dados_pf, columns=['MÊS', 'REGIÃO', 'NÚMERO DE BENEFICIARIOS ATIVO PF',
                                            'VALOR DA RECEITA PF', 'VALOR DAS DESPESAS PF'])

    # Criar a planilha com a aba "PJ"
    df_pj = pd.DataFrame(dados_pj, columns=['MÊS', 'REGIÃO', 'NÚMERO DE BENEFICIARIOS ATIVO PJ',
                                            'VALOR DA RECEITA PJ', 'VALOR DAS DESPESAS PJ'])

    # Exportar os DataFrames para um único arquivo Excel com várias abas
    nome_arquivo_saida = os.path.join(diretorio_codigo, 'planilhas_resultado_analise_pf_pj.xlsx')
    with pd.ExcelWriter(nome_arquivo_saida) as writer:
        df_pf.to_excel(writer, sheet_name='PF', index=False)
        df_pj.to_excel(writer, sheet_name='PJ', index=False)

if __name__ == '__main__':
    main()
